In [ ]:
import pickle
import numpy as np
import pandas as pd

In [ ]:
xgb_mrs_predictions_path = '/Users/jk1/temp/opsum_prediction_output/linear_72h_xgb/3M_mrs02/with_feature_aggregration/testing/test_gt_and_pred.pkl'
transformer_mrs_predictions_path = '/Users/jk1/temp/opsum_prediction_output/transformer/3M_mrs02/transformer_20230402_184459_test_set_evaluation/fold_2_test_gt_and_pred.pkl'
mlp_mrs_predictions_path = '/Users/jk1/temp/opsum_prediction_output/linear_72h_perceptron/mrs02/with_feature_aggregration_mlp/all_folds/test_gt_and_pred_cv_2.pkl'

xgb_death_predictions_path = '/Users/jk1/temp/opsum_prediction_output/linear_72h_xgb/3M_Death/testing/test_gt_and_pred.pkl'
transformer_death_predictions_path = '/Users/jk1/temp/opsum_prediction_output/transformer/3M_Death/testing/fold_1_test_gt_and_pred.pkl'
mlp_death_predictions_path = '/Users/jk1/temp/opsum_prediction_output/linear_72h_perceptron/death/feature_aggregration_mlp/testing/test_gt_and_pred_cv_2.pkl'

xgb_mimic_death_predictions_path = '/Users/jk1/temp/opsum_prediction_output/linear_72h_xgb/3M_Death/external_validation/external_validation_gt_and_pred.pkl'
transformer_mimic_death_predictions_path = '/Users/jk1/temp/opsum_prediction_output/transformer/3M_Death/external_validation/fold_1_test_gt_and_pred.pkl'
mlp_mimic_death_predictions_path = '/Users/jk1/temp/opsum_prediction_output/linear_72h_perceptron/death/feature_aggregration_mlp/external_validation/fold_2_test_gt_and_pred.pkl'

# Compare transformer and XGB

In [ ]:
from sklearn.metrics import roc_curve, auc
# Create a table with the AUC of the two models for mrs / death and mimic (with p-values)

import pandas as pd
import numpy as np
from prediction.outcome_prediction.model_comparison.de_long_comparison import delong_roc_test

def get_auc(gt, predictions):
    fpr, tpr, _ = roc_curve(gt, predictions)
    return auc(fpr, tpr)

def get_auc_and_p_value(gt, other_model_predictions, transformer_predictions):
    auc_other_model = get_auc(gt, other_model_predictions)
    auc_transformer = get_auc(gt, transformer_predictions)
    p_value = 10 ** delong_roc_test(gt, other_model_predictions, transformer_predictions)
    return auc_other_model, auc_transformer, p_value

def get_auc_and_p_value_for_outcome(outcome):
    if outcome == '3M Death':
        xgb_predictions_path = xgb_death_predictions_path
        transformer_predictions_path = transformer_death_predictions_path
    elif outcome == '3M mRS 0-2':
        xgb_predictions_path = xgb_mrs_predictions_path
        transformer_predictions_path = transformer_mrs_predictions_path

    elif outcome == 'MIMIC 3M Death':
        xgb_predictions_path = xgb_mimic_death_predictions_path
        transformer_predictions_path = transformer_mimic_death_predictions_path
    else:
        raise ValueError('Unknown outcome')

    xgb_gt, xgb_predictions = pickle.load(open(xgb_predictions_path, 'rb'))
    transformer_gt, transformer_predictions = pickle.load(open(transformer_predictions_path, 'rb'))

    auc_xgb, auc_transformer, p_value = get_auc_and_p_value(xgb_gt, xgb_predictions, transformer_predictions)
    return auc_xgb, auc_transformer, p_value

df = pd.DataFrame(columns=['Outcome', 'XGB AUC', 'Transformer AUC', 'p-value'])
for outcome in ['3M Death', '3M mRS 0-2', 'MIMIC 3M Death']:
    auc_xgb, auc_transformer, p_value = get_auc_and_p_value_for_outcome(outcome)
    df = df.append({'Outcome': outcome, 'XGB AUC': auc_xgb, 'Transformer AUC': auc_transformer, 'p-value': p_value[0][0]}, ignore_index=True)


In [ ]:
df

In [ ]:
df.to_csv('/Users/jk1/Downloads/model_comparison_xgb_transformer.csv')

# Compare MLP with transformer

In [ ]:
def get_auc_and_p_value_for_outcome_mlp(outcome):
    if outcome == '3M Death':
        mlp_predictions_path = mlp_death_predictions_path
        transformer_predictions_path = transformer_death_predictions_path
    elif outcome == '3M mRS 0-2':
        mlp_predictions_path = mlp_mrs_predictions_path
        transformer_predictions_path = transformer_mrs_predictions_path

    elif outcome == 'MIMIC 3M Death':
        mlp_predictions_path = mlp_mimic_death_predictions_path
        transformer_predictions_path = transformer_mimic_death_predictions_path
    else:
        raise ValueError('Unknown outcome')

    mlp_gt, mlp_predictions = pickle.load(open(mlp_predictions_path, 'rb'))
    transformer_gt, transformer_predictions = pickle.load(open(transformer_predictions_path, 'rb'))

    auc_mlp, auc_transformer, p_value = get_auc_and_p_value(mlp_gt, mlp_predictions, transformer_predictions)
    return auc_mlp, auc_transformer, p_value

df = pd.DataFrame(columns=['Outcome', 'MLP AUC', 'Transformer AUC', 'p-value'])
for outcome in ['3M Death', '3M mRS 0-2', 'MIMIC 3M Death']:
    auc_mlp, auc_transformer, p_value = get_auc_and_p_value_for_outcome_mlp(outcome)
    df = df.append({'Outcome': outcome, 'MLP AUC': auc_mlp, 'Transformer AUC': auc_transformer, 'p-value': p_value[0][0]}, ignore_index=True)

In [ ]:
df

In [ ]:
df.to_csv('/Users/jk1/Downloads/model_comparison_mlp_transformer.csv')